# Droplet segmentation

In [ ]:
from functools import partial
from multiprocessing import Pool
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.ndimage import distance_transform_edt
from skimage.color import label2rgb
from skimage.draw import disk
from skimage.feature import graycomatrix, graycoprops
from skimage.filters import threshold_multiotsu
from skimage.measure import regionprops_table
from skimage.segmentation import clear_border, watershed
from skimage.util import img_as_ubyte, map_array
from tifffile import imread, imwrite
from tqdm.auto import tqdm

from droplet_segmentation_tasks import get_droplets

In [ ]:
%matplotlib widget
img_dir = "../data/img"
labels_dir = "../data/labels"
img_info_file = "../data/images.csv"
droplets_file = "../data/droplets.csv"

## Image loading

In [ ]:
img_files = sorted(Path(img_dir).glob("*.tiff"))
raw_img_list = [imread(img_file) for img_file in tqdm(img_files)]  # shape: (n, t, y, x)
n = len(raw_img_list)

In [ ]:
img_info = pd.DataFrame(
    data=[
        {
            "image": img_file.name,
            "image_width": raw_img.shape[-1],
            "image_height": raw_img.shape[-2],
        }
        for img_file, raw_img in zip(img_files, raw_img_list)
    ]
)
img_info.to_csv(img_info_file, index=False)
img_info

In [ ]:
t = -1
vmin = min(np.amin(raw_img) for raw_img in raw_img_list)
vmax = max(np.amax(raw_img) for raw_img in raw_img_list)
fig, axs = plt.subplots(
    nrows=2, ncols=n, squeeze=False, height_ratios=(2, 1), figsize=(4 * n, 6), constrained_layout=True
)
for img_file, raw_img, (ax1, ax2) in zip(img_files, raw_img_list, axs.transpose()):
    ax1.set_title(f"{img_file.name}\n(t = {t})", fontsize="small")
    ax1.imshow(raw_img[t], cmap="gray", vmin=vmin, vmax=vmax)
    ax1.set_axis_off()
    ax2.set_title(f"{img_file.name}", fontsize="small")
    ax2.hist(raw_img.flatten(), bins=256, range=(vmin, vmax), density=True)
    ax2.set_xlabel("Pixel value")
    ax2.set_ylabel("Density")
plt.show()

## Image preprocessing
1. Gaussian smoothing to remove artifacts
2. Grayscale opening to fill holes in circles
3. Min-max normalization for visualization
4. Conversion to unsigned 8-bit integer range

In [ ]:
gaussian_sigma = 5
opening_radius = 10

In [ ]:
gaussian_ksize = (6 * gaussian_sigma + 1, 6 * gaussian_sigma + 1)
opening_ksize = (2 * opening_radius + 1, 2 * opening_radius + 1)
opening_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, opening_ksize)
img_list = [
    cv2.normalize(
        np.array(
            [
                cv2.morphologyEx(
                    cv2.GaussianBlur(raw_img_t, gaussian_ksize, gaussian_sigma),
                    cv2.MORPH_OPEN,
                    opening_kernel,
                )
                for raw_img_t in raw_img
            ]
        ),
        None,
        0,
        255,
        cv2.NORM_MINMAX,
    ).astype(np.uint8)
    for raw_img in tqdm(raw_img_list)
]  # shape: (n, t, y, x)

In [ ]:
t = -1
fig, axs = plt.subplots(
    nrows=2, ncols=n, squeeze=False, height_ratios=(2, 1), figsize=(4 * n, 6), constrained_layout=True
)
for img_file, img, (ax1, ax2) in zip(img_files, img_list, axs.transpose()):
    ax1.set_title(f"{img_file.name}\n(t = {t})", fontsize="small")
    ax1.imshow(img[t], cmap="gray", vmin=0, vmax=255)
    ax1.set_axis_off()
    ax2.set_title(f"{img_file.name}", fontsize="small")
    ax2.hist(img.flatten(), bins=256, range=(0, 255), density=True)
    ax2.set_xlabel("Pixel value")
    ax2.set_ylabel("Density")
plt.show()

## Circle mask generation
Three-class (outer circles, inner circles/other droplet content, background) Otsu thresholding to mask circles

In [ ]:
thresholds_list = [
    threshold_multiotsu(img, classes=3) for img in tqdm(img_list)
]  # shape: (n, 2)
edges_list = [
    np.where(img < thresholds[0], 255, 0).astype(np.uint8)
    for img, thresholds in zip(img_list, thresholds_list)
]  # shape: (n, t, y, x)

In [ ]:
t = -1
fig, axs = plt.subplots(
    nrows=2, ncols=n, squeeze=False, height_ratios=(1, 2), figsize=(4 * n, 6), constrained_layout=True
)
for img_file, img, edges, thresholds, (ax1, ax2) in zip(
    img_files, img_list, edges_list, thresholds_list, axs.transpose()
):
    ax1.set_title(img_file.name, fontsize="small")
    ax1.hist(img.flatten(), bins=256, range=(0, 255), density=True)
    ax1.axvline(thresholds[0], color="red")
    for threshold in thresholds[1:]:
        ax1.axvline(threshold, color="red", linestyle="--")
    ax1.set_xlabel("Pixel value")
    ax1.set_ylabel("Density")
    ax2.set_title(f"{img_file.name}\n(t = {t})", fontsize="small")
    ax2.imshow(edges[t], cmap="gray")
    ax2.set_axis_off()
plt.show()

## Circular droplet detection
Hough circle transform on circle masks to detect droplets

In [ ]:
hough_min_radius = 160
hough_max_radius = 200
hough_radius_step = 5
hough_min_distance = 200

In [ ]:
f = partial(
    get_droplets,
    hough_min_radius=hough_min_radius,
    hough_max_radius=hough_max_radius,
    hough_radius_step=hough_radius_step,
    hough_min_distance=hough_min_distance,
)
with Pool() as pool:
    droplets_lists = [
        pool.map(f, edges) for edges in tqdm(edges_list)
    ]  # shape: (n, t, c, 4)

In [ ]:
t = -1
fig, axs = plt.subplots(
    nrows=2, ncols=n, squeeze=False, height_ratios=(2, 1), figsize=(4 * n, 6), constrained_layout=True
)
for img_file, img, droplets_list, (ax1, ax2) in zip(
    img_files, img_list, droplets_lists, axs.transpose()
):
    ax1.set_title(f"{img_file.name}\n(t = {t})", fontsize="small")
    ax1.imshow(img[t], cmap="gray", vmin=0, vmax=255)
    last_scatter = ax1.scatter(
        droplets_list[t][:, 1],
        droplets_list[t][:, 2],
        s=150,
        c=droplets_list[t][:, 0],
        vmin=0,
        vmax=1,
    )
    for accum, cx, cy, radius in droplets_list[t]:
        ax1.add_patch(plt.Circle((cx, cy), radius=radius, color="red", fill=False))
    ax1.set_axis_off()
    ax2.set_title(img_file.name, fontsize="small")
    ax2.hist(
        np.concatenate([droplets[:, 0] for droplets in droplets_list]),
        bins=50,
        range=(-0.001, 1.001),
        density=True,
    )
    ax2.set_xlabel("Accumulator")
    ax2.set_ylabel("Density")
plt.colorbar(last_scatter, ax=axs[0].tolist(), label="Accumulator")
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12, 4), constrained_layout=True)
ax1.hist(
    np.concatenate(
        [
            droplets[:, 0]
            for droplets_list in droplets_lists
            for droplets in droplets_list
        ]
    ),
    bins=50,
    density=True,
)
ax1.set_xlabel("Accumulator")
ax1.set_ylabel("Density")
for img_file, droplets_list in zip(img_files, droplets_lists):
    droplet_counts = np.array([len(droplets) for droplets in droplets_list])
    ax2.plot(droplet_counts, label=img_file.name, linestyle="-", marker=".")
ax2.legend(loc="upper left", fontsize="small")
ax2.set_xlabel("Time")
ax2.set_ylabel("Droplets")
plt.show()

## Circular droplet filtering
Thresholding of normalized Hough accumulator score to filter droplets

In [ ]:
min_accum = 0.8

In [ ]:
filtered_droplets_lists = [
    [droplets[droplets[:, 0] >= min_accum, :] for droplets in droplets_list]
    for droplets_list in droplets_lists
]  # shape: (n, t, c, 4)

In [ ]:
t = -1
fig, axs = plt.subplots(
    nrows=2, ncols=n, squeeze=False, height_ratios=(2, 1), figsize=(4 * n, 6), constrained_layout=True
)
for img_file, img, filtered_droplets_list, (ax1, ax2) in zip(
    img_files, img_list, filtered_droplets_lists, axs.transpose()
):
    ax1.set_title(f"{img_file.name}\n(t = {t})", fontsize="small")
    ax1.imshow(img[t], cmap="gray", vmin=0, vmax=255)
    last_scatter = ax1.scatter(
        filtered_droplets_list[t][:, 1],
        filtered_droplets_list[t][:, 2],
        s=150,
        c=filtered_droplets_list[t][:, 0],
        vmin=0,
        vmax=1,
    )
    for accum, cx, cy, radius in filtered_droplets_list[t]:
        ax1.add_patch(plt.Circle((cx, cy), radius=radius, color="red", fill=False))
    ax1.set_axis_off()
    ax2.set_title(img_file.name, fontsize="small")
    ax2.hist(
        np.concatenate(
            [filtered_droplets[:, 0] for filtered_droplets in filtered_droplets_list]
        ),
        bins=50,
        range=(-0.001, 1.001),
        density=True,
    )
    ax2.set_xlabel("Accumulator")
    ax2.set_ylabel("Density")
plt.colorbar(last_scatter, ax=axs[0].tolist(), label="Accumulator")
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12, 4), constrained_layout=True)
ax1.hist(
    np.concatenate(
        [
            filtered_droplets[:, 0]
            for filtered_droplets_list in filtered_droplets_lists
            for filtered_droplets in filtered_droplets_list
        ]
    ),
    bins=100,
    range=(-0.001, 1.001),
    density=True,
)
ax1.set_xlabel("Accumulator")
ax1.set_ylabel("Density")
for img_file, filtered_droplets_list in zip(img_files, filtered_droplets_lists):
    filtered_droplet_counts = np.array(
        [len(filtered_droplets) for filtered_droplets in filtered_droplets_list]
    )
    ax2.plot(filtered_droplet_counts, label=img_file.name, linestyle="-", marker=".")
ax2.legend(loc="upper left", fontsize="small")
ax2.set_xlabel("Time")
ax2.set_ylabel("Droplets")
plt.show()

## Circular droplet refinement

Euclidean distance transform to find maximum radius of droplet

In [ ]:
refined_droplets_lists = []
for edges, filtered_droplets_list in zip(tqdm(edges_list), filtered_droplets_lists):
    refined_droplets_list = []
    for t, filtered_droplets in enumerate(filtered_droplets_list):
        mask = edges[t] != 0
        for accum, cx, cy, radius in filtered_droplets:
            rr, cc = disk((cy, cx), radius, shape=mask.shape)
            mask[rr, cc] = True
        dists = distance_transform_edt(mask)
        refined_droplets_data = []
        for accum, cx, cy, radius in filtered_droplets:
            rr, cc = disk((cy, cx), radius, shape=mask.shape)
            cur_dists = np.zeros_like(dists)
            cur_dists[rr, cc] = dists[rr, cc]
            new_cy, new_cx = np.unravel_index(np.argmax(cur_dists), cur_dists.shape)
            max_radius = cur_dists[new_cy, new_cx]
            refined_droplets_data.append([accum, new_cx, new_cy, radius, max_radius])
        refined_droplets = np.array(refined_droplets_data)
        refined_droplets_list.append(refined_droplets)
    refined_droplets_lists.append(refined_droplets_list)

In [ ]:
t = -1
fig, axs = plt.subplots(
    nrows=2, ncols=n, squeeze=False, height_ratios=(2, 1), figsize=(4 * n, 6), constrained_layout=True
)
for img_file, img, refined_droplets_list, (ax1, ax2) in zip(
    img_files, img_list, refined_droplets_lists, axs.transpose()
):
    ax1.set_title(f"{img_file.name}\n(t = {t})", fontsize="small")
    ax1.imshow(img[t], cmap="gray", vmin=0, vmax=255)
    last_scatter = ax1.scatter(
        refined_droplets_list[t][:, 1],
        refined_droplets_list[t][:, 2],
        s=150,
        c=refined_droplets_list[t][:, 0],
        vmin=0,
        vmax=1,
    )
    for accum, cx, cy, radius, max_radius in refined_droplets_list[t]:
        ax1.add_patch(plt.Circle((cx, cy), radius=max_radius, color="red", fill=False))
    ax1.set_axis_off()
    ax2.set_title(img_file.name, fontsize="small")
    ax2.hist(
        np.concatenate(
            [refined_droplets[:, 0] for refined_droplets in refined_droplets_list if len(refined_droplets) > 0]
        ),
        bins=50,
        range=(-0.001, 1.001),
        density=True,
    )
    ax2.set_xlabel("Accumulator")
    ax2.set_ylabel("Density")
plt.colorbar(last_scatter, ax=axs[0].tolist(), label="Accumulator")
plt.show()

## Droplet labels generation

Border-free droplet segmentation using maximum radius-constrained watershed

In [ ]:
labels_list = []
labeled_droplets_lists = []
Path(labels_dir).mkdir(exist_ok=True)
for img_file, edges, refined_droplets_list in zip(
    tqdm(img_files), edges_list, refined_droplets_lists
):
    labeled_droplets_list = []
    labels = np.zeros_like(edges, dtype=np.uint8)
    for t, refined_droplets in enumerate(refined_droplets_list):
        labeled_droplets_data = []
        mask = edges[t] != 0
        max_mask = np.zeros_like(mask)
        markers = np.zeros_like(edges[t], dtype=np.uint8)
        for label, (accum, cx, cy, radius, max_radius) in enumerate(
            refined_droplets, start=1
        ):
            rr, cc = disk((cy, cx), radius, shape=mask.shape)
            mask[rr, cc] = True
            max_rr, max_cc = disk((cy, cx), max_radius, shape=max_mask.shape)
            max_mask[max_rr, max_cc] = True
            markers[int(cy), int(cx)] = label
            labeled_droplets_data.append([accum, cx, cy, radius, max_radius, label])
        labeled_droplets = np.array(labeled_droplets_data)
        dists = distance_transform_edt(mask)
        current_labels = watershed(-dists, markers=markers, mask=max_mask)
        current_labels = clear_border(current_labels)
        unique_labels = np.unique(current_labels[current_labels != 0])
        labeled_droplets = labeled_droplets[
            np.isin(labeled_droplets[:, -1], unique_labels),
            :,
        ]
        labeled_droplets_list.append(labeled_droplets)
        labels[t] = current_labels
    labels_list.append(labels)
    labeled_droplets_lists.append(labeled_droplets_list)
    imwrite(Path(labels_dir) / img_file.name, labels)

In [ ]:
t = -1
fig, axs = plt.subplots(ncols=n, squeeze=False, figsize=(4 * n, 4), constrained_layout=True)
for img_file, img, labels, ax in zip(img_files, img_list, labels_list, axs.flat):
    ax.set_title(f"{img_file.name}\n(t = {t})", fontsize="small")
    ax.imshow(label2rgb(labels[t], image=img[t], alpha=0.5))
    ax.set_axis_off()
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12, 4), constrained_layout=True)
ax1.hist(
    np.concatenate(
        [
            labeled_droplets[:, 0]
            for labeled_droplets_list in labeled_droplets_lists
            for labeled_droplets in labeled_droplets_list
        ]
    ),
    bins=100,
    range=(-0.001, 1.001),
    density=True,
)
ax1.set_xlabel("Accumulator")
ax1.set_ylabel("Density")
for img_file, labeled_droplets_list in zip(img_files, labeled_droplets_lists):
    labeled_droplets_counts = np.array(
        [len(labeled_droplets) for labeled_droplets in labeled_droplets_list]
    )
    ax2.plot(labeled_droplets_counts, label=img_file.name, linestyle="-", marker=".")
ax2.legend(loc="upper left", fontsize="small")
ax2.set_xlabel("Time")
ax2.set_ylabel("Droplets")
plt.show()

## Droplet quantification

Compute mean intensities and GLCM texture features for all filtered, refined, and labeled droplets

In [ ]:
texture_bins = 64
texture_dists = [4]
texture_angles = [0]
texture_props = [
    "contrast",
    "dissimilarity",
    "homogeneity",
    "ASM",
    "energy",
    "correlation",
]

In [ ]:
def texture_properties(labels, img):
    if min(labels.shape) < max(texture_dists) * 2 + 1:
        return [np.nan] * len(texture_props)
    r = min(img.shape) / 4
    cy, cx = img.shape[0] / 2, img.shape[1] / 2
    glcm = graycomatrix(
        img[int(cy - r):int(cy + r), int(cx - r):int(cx + r)], 
        texture_dists,
        texture_angles,
        symmetric=True,
        normed=True,
    )
    return [graycoprops(glcm, texture_prop) for texture_prop in texture_props]

In [ ]:
droplets_info = pd.concat(
    {
        (img_file.name, t): pd.DataFrame(
            data=labeled_droplets[:, :-1],
            columns=["accum", "center_x", "center_y", "radius", "max_radius"],
            index=labeled_droplets[:, -1].astype(int),
        )
        for img_file, labeled_droplets_list in zip(img_files, labeled_droplets_lists)
        for t, labeled_droplets in enumerate(labeled_droplets_list)
    },
    names=["image", "time", "label"],
)
droplets_info.reset_index(inplace=True)

In [ ]:
droplets_intensity_info = pd.concat(
    {
        (img_file.name, t): pd.DataFrame(
            data=regionprops_table(
                labels_t,
                intensity_image=img_t,
                properties=["label", "intensity_mean"],
            ),
        ).set_index("label")
        for img_file, img, labels in zip(img_files, img_list, labels_list)
        for t, (img_t, labels_t) in enumerate(zip(img, labels))
    },
    names=["image", "time", "label"]
)
droplets_intensity_info.reset_index(inplace=True)
droplets_info = pd.merge(
    droplets_info,
    droplets_intensity_info,
    on=["image", "time", "label"]
)

In [ ]:
droplets_texture_info = pd.concat(
    {
        (img_file.name, t): pd.DataFrame(
            data=regionprops_table(
                labels_t,
                intensity_image=img_as_ubyte(raw_img_t // int(256 / texture_bins)),
                properties=["label"],
                extra_properties=[texture_properties],
            ),
        ).set_index("label")
        for img_file, raw_img, labels in zip(img_files, raw_img_list, labels_list)
        for t, (raw_img_t, labels_t) in enumerate(zip(raw_img, labels))
    },
    names=["image", "time", "label"]
)
droplets_texture_info.columns = texture_props
droplets_texture_info.reset_index(inplace=True)
droplets_info = pd.merge(
    droplets_info,
    droplets_texture_info,
    on=["image", "time", "label"]
)

In [ ]:
droplets_info.to_csv(droplets_file, index=False)
droplets_info

In [ ]:
t = -1
fig, axs = plt.subplots(
    nrows=n,
    ncols=len(texture_props),
    squeeze=False,
    figsize=(3 * len(texture_props), 3 * n),
    constrained_layout=True,
)
for row, (img_file, img, labels, row_axs) in enumerate(
    zip(img_files, img_list, labels_list, axs)
):
    df = droplets_info[droplets_info["image"] == img_file.name]
    df = df[df["time"] == (droplets_info["time"].max() + 1 + t if t < 0 else t)]
    for col, (texture_prop, ax) in enumerate(zip(texture_props, row_axs)):
        if row == 0:
            ax.set_title(texture_prop, fontsize="small")
        if col == 0:
            ax.set_ylabel(f"{img_file.name}\n(t = {t})", fontsize="small")
        ax.imshow(img[t], cmap="gray", vmin=0, vmax=255)
        ax.imshow(
            map_array(labels[t], df["label"].to_numpy(), df[texture_prop].to_numpy()),
            alpha=0.3,
            cmap="coolwarm",
            vmin=droplets_info[texture_prop].quantile(0.05),
            vmax=droplets_info[texture_prop].quantile(0.95),
        )
        ax.tick_params(axis="both", which="both", length=0, labelsize=0)
plt.show()

In [ ]:
fig, axs = plt.subplots(
    nrows=n,
    ncols=len(texture_props),
    squeeze=False,
    figsize=(3 * len(texture_props), 3 * n),
    constrained_layout=True,
)
for row, (img_file, img, labels, row_axs) in enumerate(
    zip(img_files, img_list, labels_list, axs)
):
    df = droplets_info.loc[(droplets_info["image"] == img_file.name)]
    for col, (texture_prop, ax) in enumerate(zip(texture_props, row_axs)):
        if row == 0:
            ax.set_title(texture_prop, fontsize="small")
        if col == 0:
            ax.set_ylabel(img_file.name)
        ax.scatter(df["time"], df[texture_prop])
        ax.set_xlabel("time")
plt.show()

In [ ]:
# texture_prop = "dissimilarity"
# texture_threshold = 1.5
# max_time = droplets_info.time.max() + 1
# fig, axs = plt.subplots(
#     nrows=max_time, ncols=n, squeeze=False, figsize=(2 * n, 2 * max_time), constrained_layout=True
# )
# for col, (img_file, img, labels) in enumerate(zip(img_files, img_list, labels_list)):
#     for row, (img_t, labels_t) in enumerate(zip(img, labels)):
#         df = droplets_info.loc[
#             (droplets_info["image"] == img_file.name) & (droplets_info["time"] == row)
#         ]
#         axs[row, col].set_title(f"{img_file.name}\n(t = {row})", fontsize="small")
#         axs[row, col].imshow(img_t, cmap="gray", vmin=0, vmax=255)
#         axs[row, col].imshow(
#             map_array(labels_t, df["label"].to_numpy(), df[texture_prop].to_numpy() > texture_threshold),
#             alpha=0.3,
#             cmap="coolwarm",
#             )
#     for ax in axs.flatten():
#         ax.set_axis_off()
# plt.show()

## Notes

Other approaches/failed attempts:
- Edge detection instead of thresholding: inner/outer radii vary with circle thickness
- Contour detection + ellipse fitting instead of Hough transform: thinning yields connected contours
- Learning-based segmentation using Ilastik: random forests fail to discriminate droplet and background classes

In [ ]:
# labels_list = []
# for edges, filtered_droplets_list in zip(edges_list, filtered_droplets_lists):
#     labels = np.zeros_like(edges, dtype=np.uint8)
#     for t, filtered_droplets in enumerate(filtered_droplets_list):
#         mask = edges[t] != 0
#         markers = np.zeros_like(edges[t], dtype=np.uint8)
#         for i, (accum, cx, cy, radius) in enumerate(filtered_droplets):
#             rr, cc = disk((cy, cx), radius, shape=mask.shape)
#             mask[rr, cc] = True
#             markers[int(cy), int(cx)] = i + 1
#         dists = distance_transform_edt(edges[t] == 0)
#         labels[t] = watershed(-dists, markers=markers, mask=mask)
#     labels_list.append(labels)

In [ ]:
# min_ellipse_area = 30000
# max_ellipse_eccentricity = 0.7
# max_ellipse_match_difference = 0.05

# ellipse_areas = []
# ellipse_eccentricities = []
# ellipse_match_differences = []
# masks = np.zeros_like(thres_imgs8, dtype=np.uint16)
# for i, thres_img8 in enumerate(tqdm(thres_imgs8)):
#     contours, hierarchy = cv2.findContours(
#         thres_img8, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE
#     )
#     next_label = int(np.max(masks[i])) + 1
#     for j, (contour, h) in enumerate(zip(contours, hierarchy[0])):
#         h_next, h_previous, h_child, h_parent = h
#         if h_parent >= 0 and len(contour) >= 5:
#             (cx, cy), (a, b), angle = cv2.fitEllipse(contour)
#             if a > 0 and b > 0:
#                 assert a <= b
#                 ellipse_area = 0.25 * np.pi * a * b
#                 ellipse_eccentricity = (1 - (a / b) ** 2) ** 0.5
#                 ellipse_contour = cv2.ellipse2Poly(
#                     (int(cx), int(cy)), (int(a), int(b)), int(angle), 0, 360, 1
#                 )
#                 ellipse_match_difference = cv2.matchShapes(
#                     contour, ellipse_contour, cv2.CONTOURS_MATCH_I1, 0.0
#                 )
#                 if (
#                     ellipse_area >= min_ellipse_area
#                     and ellipse_eccentricity <= max_ellipse_eccentricity
#                     and ellipse_match_difference <= max_ellipse_match_difference
#                 ):
#                     masks[i] = cv2.drawContours(
#                         masks[i], contours, j, next_label, thickness=cv2.FILLED
#                     )
#                     next_label += 1
#                 ellipse_areas.append(ellipse_area)
#                 ellipse_eccentricities.append(ellipse_eccentricity)
#                 ellipse_match_differences.append(ellipse_match_difference)
# ellipse_areas = np.array(ellipse_areas)
# ellipse_eccentricities = np.array(ellipse_eccentricities)
# ellipse_match_differences = np.array(ellipse_match_differences)

# t = 0
# plt.figure("Mask")
# plt.title(f"t = {t}")
# plt.imshow(masks[t], cmap="gray")
# plt.axis("off")
# plt.show()

# m1 = ellipse_areas >= min_ellipse_area
# m2 = ellipse_eccentricities <= max_ellipse_eccentricity
# m3 = ellipse_match_differences <= max_ellipse_match_difference

# fig, (ax1, ax2, ax3) = plt.subplots(
#     nrows=1,
#     ncols=3,
#     squeeze=False,
#     figsize=(
#         3 * plt.rcParams["figure.figsize"][0],
#         plt.rcParams["figure.figsize"][1],
#     ),
# )
# fig.suptitle("Ellipses")

# ax1.hist(
#     ellipse_areas[m2 & m3], bins=50, range=(0, 50000), density=True, color="gray"
# )
# ax1.axvline(min_ellipse_area, color="blue")
# ax1.set_xlabel("Area")
# ax1.set_ylabel("Density")

# ax2.hist(
#     ellipse_eccentricities[m1 & m3], bins=50, range=(0, 1), density=True, color="gray"
# )
# ax2.axvline(max_ellipse_eccentricity, color="blue")
# ax2.set_xlabel("Eccentricity")
# ax2.set_ylabel("Density")

# ax3.hist(
#     ellipse_match_differences[m1 & m2],
#     bins=100,
#     range=(0, 0.1),
#     density=True,
#     color="gray",
# )
# ax3.axvline(max_ellipse_match_difference, color="blue")
# ax3.set_xlabel("Match difference")
# ax3.set_ylabel("Density")

# plt.show()